In [1]:
import torch
import torch.nn as nn
import torch.optim as opt
import torchvision
from torchvision import transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
from PIL import Image
import os.path as osp
import numpy as np
from numpy import random
from numpy.random import uniform
from math import sqrt
import PIL
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from random import shuffle
import time
import math
import argparse
import os
import warnings
import copy
import cv2
warnings.filterwarnings("ignore")
import imageio


## UNet

#### Padding and Resize images

In [2]:
def transfrom(data):
    width=data.size[0]
    hight=data.size[1]
    if width>hight:
        pad=(width-hight)//2
        data=transforms.functional.pad(data,padding=(0,pad))
    elif width<hight:
        pad=(hight-width)//2
        data=transforms.functional.pad(data,padding=(pad,0))  
    data=transforms.functional.resize(data,(512,512)) 
    data=transforms.functional.to_tensor(data).float()
    return data

#### The UNet Model

In [3]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3,padding=1,bias=False),
            nn.BatchNorm2d(out_ch), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3,padding=1,bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
 
    def forward(self, input):        
        return self.conv(input)

In [4]:
class UNet(nn.Module):
    def __init__(self,colordim =1):
        super(UNet, self).__init__()
        self.conv1=DoubleConv(3,32)
        self.conv2=DoubleConv(32,64)
        self.conv3=DoubleConv(64,128)
        self.conv4=DoubleConv(128,256)
        
        self.conv5=DoubleConv(256,512)
        self.upconv5=nn.Conv2d(512, 256, 1)
        
        self.conv6=DoubleConv(512,256)
        self.upconv6=nn.Conv2d(256, 128, 1)
        self.conv7=DoubleConv(256,128)
        self.upconv7=nn.Conv2d(128,64, 1)
        self.conv8=DoubleConv(128,64)
        self.upconv8=nn.Conv2d(64,32, 1)
        self.conv9=DoubleConv(64,32)
        
        self.maxpool=nn.MaxPool2d(2, stride=2)
        self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)

        self.output=nn.Sequential(
            nn.Conv2d(32, colordim, 1),
        )
        self._initialize_weights()
    
    def forward(self, x):
        x1=self.conv1(x)
        x2=self.conv2(self.maxpool(x1))
        x3=self.conv3(self.maxpool(x2))
        x4=self.conv4(self.maxpool(x3))
        upx=self.conv5(self.maxpool(x4))
        
        upx=self.upconv5(self.upsample(upx))     

        upx=self.conv6(torch.cat((x4, upx), 1))
        upx=self.upconv6(self.upsample(upx))   

        upx=self.conv7(torch.cat((x3, upx), 1))
        upx=self.upconv7(self.upsample(upx))  

        upx=self.conv8(torch.cat((x2, upx), 1))
        upx=self.upconv8(self.upsample(upx)) 

        upx=self.conv9(torch.cat((x1, upx), 1))
        upx=self.output(upx)
        
        return upx
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()        
        
        



## Get Face Information

#### Load the Model

In [5]:
unet = UNet(colordim=20).cuda()
unet.load_state_dict(torch.load('../input/real-time-unet-weights/s_30000_20.pkl'))
unet.eval()

UNet(
  (conv1): DoubleConv(
    (conv): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv2): DoubleConv(
    (conv): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv3): DoubleConv(
    (conv): Sequen

#### Get the UNet outputs and save as git file

In [22]:
video_capture = cv2.VideoCapture('../input/morden-train/5.mp4')
process_this_frame = True
face_infos=[]
fram_count=0
frames=[]
with torch.no_grad():    
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()
        if not ret:
            break
        small_frame=frame
        rgb_small_frame = small_frame[:, :, ::-1]
        rgb_small_frame=transfrom(Image.fromarray(rgb_small_frame))   
        output=unet(torch.unsqueeze(rgb_small_frame,dim=0).cuda()).to(torch.float64)
        output=torch.max(output,1)[1]
        output=output.cpu().detach().numpy()[0]
        frames.append(output)
imageio.mimsave('try.gif', frames, 'GIF', duration = 0.05)

    